In [1]:
#importing required libraries
import os
import shutil
import time

import folium
import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
./Data#reading 2017 data into respective dataframes
df1 = pd.read_csv('../Data/2017/Divvy_Trips_2017_Q1.csv')
df2 = pd.read_csv('../Data/2017/Divvy_Trips_2017_Q2.csv')
df3 = pd.read_csv('../Data/2017/Divvy_Trips_2017_Q3.csv')
df4 = pd.read_csv('../Data/2017/Divvy_Trips_2017_Q4.csv')

In [3]:
#Concatenating the dataframes into a single dataframe
frames = [df1,df2,df3,df4]
dfbike = pd.concat(frames)
dfbike.reset_index(drop=True,inplace=True)
dfbike.head(5)

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,13518905,3/31/2017 23:59:07,4/1/2017 00:13:24,5292,857,66,Clinton St & Lake St,171,May St & Cullerton St,Subscriber,Male,1989.0
1,13518904,3/31/2017 23:56:25,4/1/2017 00:00:21,4408,236,199,Wabash Ave & Grand Ave,26,McClurg Ct & Illinois St,Subscriber,Male,1990.0
2,13518903,3/31/2017 23:55:33,4/1/2017 00:01:21,696,348,520,Greenview Ave & Jarvis Ave,432,Clark St & Lunt Ave,Subscriber,Female,1979.0
3,13518902,3/31/2017 23:54:46,3/31/2017 23:59:34,4915,288,110,Dearborn St & Erie St,142,McClurg Ct & Erie St,Subscriber,Male,1985.0
4,13518901,3/31/2017 23:53:33,4/1/2017 00:00:28,4247,415,327,Sheffield Ave & Webster Ave,331,Halsted St & Blackhawk St (*),Subscriber,Female,1989.0


In [4]:
dfbike.columns

Index(['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear'],
      dtype='object')

### Saving all the station names in a list "allSt17"

#### Comparing with 2016 data and only adding the stations which are new into a list "added17"
#### Also, checking for the stations that have been in 2016 but not in this year's data and adding it to "removed16"

In [5]:
allSt17 = list(dfbike['from_station_name'].unique())
len(allSt17)

593

In [6]:
allSt16 = ['Ravenswood Ave & Lawrence Ave',
 'Damen Ave & Chicago Ave',
 'Ashland Ave & Chicago Ave',
 'Broadway & Cornelia Ave',
 'Lake Shore Dr & Belmont Ave',
 'Sheffield Ave & Fullerton Ave',
 'Clark St & Wellington Ave',
 'MLK Jr Dr & 47th St',
 'Mies van der Rohe Way & Chestnut St',
 'Millennium Park',
 'Calumet Ave & 33rd St',
 'State St & Kinzie St',
 'Ellis Ave & 58th St',
 'Kingsbury St & Kinzie St',
 'Damen Ave & Charleston St',
 'Lincoln Ave & Fullerton Ave',
 'Larrabee St & Oak St',
 'Franklin St & Chicago Ave',
 'Canal St & Madison St',
 'Lake Shore Dr & Wellington Ave',
 'Clark St & Bryn Mawr Ave',
 'Winchester Ave & Elston Ave',
 'Lincoln Ave & Belmont Ave',
 'Campbell Ave & Montrose Ave',
 'Ellis Ave & 55th St',
 'Clinton St & Lake St',
 'California Ave & Francis Pl',
 'State St & Harrison St',
 'Southport Ave & Wellington Ave',
 'Morgan St & 31st St',
 'Wentworth Ave & Archer Ave',
 'Clifton Ave & Lawrence Ave',
 'Lincoln Ave & Belle Plaine Ave',
 'Wells St & Elm St',
 'Desplaines St & Kinzie St',
 'Broadway & Sheridan Rd',
 'Dayton St & North Ave',
 'California Ave & North Ave',
 'Rush St & Hubbard St',
 'Troy St & Elston Ave',
 'Blackstone Ave & Hyde Park Blvd',
 'Wells St & Huron St',
 'Halsted St & Polk St',
 'Field Blvd & South Water St',
 'Manor Ave & Leland Ave',
 'Sheridan Rd & Loyola Ave',
 'Clark St & Elm St',
 'Mies van der Rohe Way & Chicago Ave',
 'Greenview Ave & Diversey Pkwy',
 'Clark St & Congress Pkwy',
 'California Ave & Milwaukee Ave',
 'University Ave & 57th St',
 'Lincoln Ave & Diversey Pkwy',
 'Damen Ave & Leland Ave',
 'Southport Ave & Roscoe St',
 'Southport Ave & Irving Park Rd',
 'Morgan St & Lake St',
 'Clarendon Ave & Leland Ave',
 'Lake Park Ave & 53rd St',
 'Woodlawn Ave & 55th St',
 'Sheridan Rd & Montrose Ave',
 'Western Ave & Walton St',
 'Damen Ave & Melrose Ave',
 'Wells St & Concord Ln',
 'Clinton St & Madison St',
 'Damen Ave & Division St',
 'Wilton Ave & Belmont Ave',
 'Broadway & Belmont Ave',
 'Cornell Ave & Hyde Park Blvd',
 'Broadway & Wilson Ave',
 'LaSalle St & Illinois St',
 'Racine Ave & 18th St',
 'Halsted St & 18th St',
 'Lake Park Ave & 56th St',
 'Green St & Madison St',
 'Michigan Ave & Oak St',
 'Racine Ave & Belmont Ave',
 'St. Clair St & Erie St',
 'Wabash Ave & 8th St',
 'Michigan Ave & Jackson Blvd',
 'Clark St & Lake St',
 'Racine Ave & Wrightwood Ave',
 'Kedzie Ave & Leland Ave',
 'Clark St & Wrightwood Ave',
 'Michigan Ave & 18th St',
 'Damen Ave & Pierce Ave',
 'Green St & Randolph St',
 'Wilton Ave & Diversey Pkwy',
 'Lake Shore Dr & Monroe St',
 'Ogden Ave & Chicago Ave',
 'Lincoln Ave & Waveland Ave',
 'Wacker Dr & Washington St',
 'Desplaines St & Randolph St',
 'Orleans St & Ohio St',
 'LaSalle St & Washington St',
 'Southport Ave & Wrightwood Ave',
 'Clark St & Grace St',
 'State St & Randolph St',
 'Southport Ave & Clark St',
 'Clifton Ave & Armitage Ave',
 'Columbus Dr & Randolph St',
 'Desplaines St & Jackson Blvd',
 'Clark St & Chicago Ave',
 'Halsted St & Archer Ave',
 'Clark St & Armitage Ave',
 'Clark St & Randolph St',
 'Larrabee St & Division St',
 'Eckhart Park',
 'Clinton St & Roosevelt Rd',
 'Ashland Ave & Division St',
 'Campbell Ave & Fullerton Ave',
 'Sedgwick St & North Ave',
 'Theater on the Lake',
 'Morgan St & Polk St',
 'Broadway & Berwyn Ave',
 'Talman Ave & Addison St',
 'Sheffield Ave & Kingsbury St',
 'Western Ave & Winnebago Ave',
 'Southport Ave & Waveland Ave',
 'Dearborn St & Adams St',
 'Morgan St & 18th St',
 'Sheridan Rd & Buena Ave',
 'Milwaukee Ave & Wabansia Ave',
 'Ashland Ave & Lake St',
 'Sheffield Ave & Addison St',
 'Halsted St & Blackhawk St (*)',
 'Stave St & Armitage Ave',
 'Artesian Ave & Hubbard St',
 'Broadway & Thorndale Ave',
 'Franklin St & Jackson Blvd',
 'State St & Van Buren St',
 'Larrabee St & Webster Ave',
 'Lakeview Ave & Fullerton Pkwy',
 'Fairbanks Ct & Grand Ave',
 'Wabash Ave & Roosevelt Rd',
 'Dearborn St & Erie St',
 'Michigan Ave & Washington St',
 'Peoria St & Jackson Blvd',
 'Daley Center Plaza',
 'Wabash Ave & Adams St',
 'Wolcott Ave & Polk St',
 'Michigan Ave & Pearson St',
 'Marshfield Ave & Cortland St',
 'Halsted St & Wrightwood Ave',
 'Calumet Ave & 21st St',
 'Clark St & Lincoln Ave',
 'Stony Island Ave & 67th St',
 'Halsted St & Maxwell St',
 'Calumet Ave & 18th St',
 'May St & Randolph St',
 'Financial Pl & Congress Pkwy',
 '900 W Harrison St',
 'Wabash Ave & Grand Ave',
 'Southport Ave & Clybourn Ave',
 'Lake Shore Dr & Diversey Pkwy',
 'Sheffield Ave & Wrightwood Ave',
 'Clark St & Winnemac Ave',
 'Leavitt St & Armitage Ave',
 'LaSalle St & Jackson Blvd',
 'Western Ave & Division St',
 'Michigan Ave & Madison St',
 'Damen Ave & Wellington Ave',
 'Indiana Ave & Roosevelt Rd',
 'Noble St & Milwaukee Ave',
 'Bissell St & Armitage Ave',
 'Halsted St & Willow St',
 'Broadway & Barry Ave',
 'Sedgwick St & Webster Ave',
 'Ellis Ave & 60th St',
 'Franklin St & Lake St',
 'Ogden Ave & Race Ave',
 'Cityfront Plaza Dr & Pioneer Ct',
 'Rush St & Superior St',
 'Sheridan Rd & Irving Park Rd',
 'Lake Shore Dr & North Blvd',
 'Rockwell St & Eastwood Ave',
 'Ashland Ave & Grace St',
 'Canal St & Taylor St',
 'Ashland Ave & Wellington Ave',
 'Cannon Dr & Fullerton Ave',
 'Ashland Ave & 21st St',
 'Dearborn Pkwy & Delaware Pl',
 'Lakefront Trail & Bryn Mawr Ave',
 'Damen Ave & Madison St',
 'Kedzie Ave & Montrose Ave',
 'Michigan Ave & 14th St',
 'Broadway & Waveland Ave',
 'Orleans St & Elm St (*)',
 'Stony Island Ave & 64th St',
 'Lake Park Ave & 47th St',
 'Ravenswood Ave & Montrose Ave (*)',
 'Western Ave & Leland Ave',
 'Milwaukee Ave & Rockwell St',
 'Wells St & Polk St',
 'Racine Ave & Congress Pkwy',
 'Damen Ave & Coulter St',
 'State St & Pearson St',
 'Loomis St & Lexington St',
 'Clarendon Ave & Gordon Ter',
 'Streeter Dr & Grand Ave',
 'Shore Dr & 55th St',
 'Aberdeen St & Monroe St',
 'Jefferson St & Monroe St',
 'Kedzie Ave & Palmer Ct',
 'Clark St & Leland Ave',
 'Damen Ave & Cortland St',
 'Clinton St & 18th St',
 'Larrabee St & Kingsbury St',
 'Clinton St & Washington Blvd',
 'Clark St & North Ave',
 'Logan Blvd & Elston Ave',
 'Aberdeen St & Jackson Blvd',
 'Lincoln Ave & Roscoe St',
 'Carpenter St & Huron St',
 'Stockton Dr & Wrightwood Ave',
 'Orleans St & Merchandise Mart Plaza',
 'Canal St & Monroe St (*)',
 'Franklin St & Quincy St',
 'Wood St & Division St',
 'Western Ave & 28th St',
 'Larrabee St & North Ave',
 'Ashland Ave & Blackhawk St',
 'Halsted St & Roosevelt Rd',
 'California Ave & Montrose Ave',
 'Dusable Harbor',
 'Broadway & Ridge Ave',
 'Ritchie Ct & Banks St',
 'Loomis St & Taylor St (*)',
 'Damen Ave & Clybourn Ave',
 'Kingsbury St & Erie St',
 'Racine Ave & Fullerton Ave',
 'Canal St & Adams St',
 'Clinton St & Polk St (*)',
 'Dearborn St & Monroe St',
 'California Ave & 23rd Pl',
 'Clybourn Ave & Division St',
 'Ashland Ave & 13th St',
 'California Ave & Altgeld St',
 'Sheffield Ave & Webster Ave',
 'Wood St & Milwaukee Ave',
 'Wells St & Evergreen Ave',
 'Humboldt Blvd & Armitage Ave',
 'Hermitage Ave & Polk St',
 'Ogden Ave & Congress Pkwy',
 'Franklin St & Monroe St',
 'Leavitt St & North Ave',
 'Lincoln Ave & Winona St',
 'Ashland Ave & Harrison St',
 'DuSable Museum',
 'Ashland Ave & Archer Ave',
 'Greenview Ave & Fullerton Ave',
 'Emerald Ave & 31st St',
 'Pine Grove Ave & Waveland Ave',
 'Racine Ave & 15th St',
 'California Ave & Division St',
 'Lincoln Ave & Leavitt St',
 'Sedgwick St & Huron St',
 'Federal St & Polk St',
 'Larrabee St & Menomonee St',
 'Michigan Ave & Lake St',
 'Clark St & Lunt Ave',
 'Blue Island Ave & 18th St',
 'Clark St & Elmdale Ave',
 'Paulina Ave & North Ave',
 'Halsted St & Roscoe St',
 'Stetson Ave & South Water St',
 'Canal St & Jackson Blvd',
 'Wabash Ave & Wacker Pl',
 'Leavitt St & Archer Ave',
 'California Ave & Byron St',
 'LaSalle St & Adams St',
 'Michigan Ave & Congress Pkwy',
 'Clark St & Montrose Ave',
 'Ada St & Washington Blvd',
 'Ashland Ave & Grand Ave',
 'Southport Ave & Belmont Ave',
 'State St & 33rd St',
 'Pine Grove Ave & Irving Park Rd',
 'Calumet Ave & 51st St',
 'Ashland Ave & Wrightwood Ave',
 'Kedzie Ave & Milwaukee Ave',
 'Damen Ave & Foster Ave',
 'Sheffield Ave & Wellington Ave',
 'Clark St & 9th St (AMLI)',
 'Monticello Ave & Irving Park Rd',
 'Hampden Ct & Diversey Pkwy',
 'Francisco Ave & Foster Ave',
 'Kedzie Ave & Foster Ave',
 'State St & 19th St',
 'Wood St & Grand Ave',
 'May St & Taylor St',
 'May St & Fulton St',
 'Marine Dr & Ainslie St',
 'Lake Shore Dr & Ohio St',
 'Sangamon St & Washington Blvd (*)',
 'Clark St & Berwyn Ave',
 'Adler Planetarium',
 'Broadway & Argyle St',
 'Kimbark Ave & 53rd St',
 'Loomis St & Jackson Blvd',
 'Ashland Ave & Belle Plaine Ave',
 'McClurg Ct & Erie St',
 'Ravenswood Ave & Irving Park Rd',
 'Halsted St & Dickens Ave',
 'Sheffield Ave & Willow St',
 'State St & 35th St',
 'Michigan Ave & Balbo Ave',
 'Union Ave & Grand Ave',
 'Drake Ave & Addison St',
 'Kedzie Ave & Bryn Mawr Ave',
 'Paulina St & 18th St',
 'Burnham Harbor',
 'California Ave & Fletcher St',
 'Dorchester Ave & 63rd St',
 'Wells St & Walton St',
 'Wabash Ave & 16th St',
 'Kimball Ave & Belmont Ave',
 'Shields Ave & 31st St',
 'Lincoln Ave & Addison St',
 'Indiana Ave & 31st St',
 'Ogden Ave & Roosevelt Rd',
 'Damen Ave & Sunnyside Ave',
 'Drake Ave & Fullerton Ave',
 'Albany Ave & Bloomingdale Ave',
 'Washtenaw Ave & 15th St (*)',
 'Clinton St & Tilden St',
 'Spaulding Ave & Armitage Ave',
 'Ravenswood Ave & Berteau Ave',
 'Clarendon Ave & Junior Ter',
 'Sedgwick St & Schiller St',
 'Montrose Harbor',
 'Damen Ave & Augusta Blvd',
 'Museum Campus',
 'Leavitt St & Lawrence Ave',
 'St. Louis Ave & Balmoral Ave',
 'Christiana Ave & Lawrence Ave',
 'Halsted St & 37th St',
 'Wentworth Ave & 24th St',
 'Larrabee St & Armitage Ave',
 'Wood St & Taylor St',
 'Cottage Grove Ave & 47th St',
 'Sheridan Rd & Lawrence Ave',
 'Damen Ave & Grand Ave',
 'Canal St & Harrison St',
 'Morgan Ave & 14th Pl',
 'McClurg Ct & Illinois St',
 'Dorchester Ave & 49th St',
 'Jeffery Blvd & 67th St',
 'Wentworth Ave & 33rd St',
 'California Ave & Lake St',
 'Shedd Aquarium',
 'Central Park Ave & Elbridge Ave',
 'Richmond St & Diversey Ave',
 'Elston Ave & Wabansia Ave',
 'Campbell Ave & North Ave',
 'Rhodes Ave & 32nd St',
 'Western Ave & Roscoe St',
 'Leavitt St & Addison St',
 'Harper Ave & 59th St',
 'Clark St & Schreiber Ave',
 'Ellis Ave & 53rd St',
 'May St & Cullerton St',
 'Keeler Ave & Irving Park Rd',
 'Rush St & Cedar St',
 'Kedzie Ave & Lake St',
 'MLK Jr Dr & 29th St',
 'Clark St & Columbia Ave',
 'Glenwood Ave & Morse Ave',
 'Halsted St & North Branch St',
 'Wells St & 19th St',
 'Claremont Ave & Hirsch St',
 'Broadway & Granville Ave',
 'Sawyer Ave & Irving Park Rd',
 'McCormick Place',
 'Fort Dearborn Dr & 31st St',
 'Greenwood Ave & 47th St',
 'California Ave & 26th St',
 'Racine Ave & 13th St',
 'Drake Ave & Montrose Ave',
 'Ashland Ave & Augusta Blvd',
 'Museum of Science and Industry',
 'Paulina St & Montrose Ave',
 'Emerald Ave & 28th St',
 'Sheridan Rd & Greenleaf Ave',
 '63rd St Beach',
 'Kedzie Ave & Chicago Ave',
 'South Shore Dr & 71st St',
 'Washtenaw Ave & Lawrence Ave',
 'Lake Park Ave & 35th St',
 'Racine Ave & 35th St',
 '2112 W Peterson Ave',
 'Avers Ave & Belmont Ave',
 'Keystone Ave & Montrose Ave',
 'Kedzie Ave & Roosevelt Rd',
 'Loomis St & Archer Ave',
 'Prairie Ave & Garfield Blvd',
 'Western Ave & Monroe St',
 'Halsted St & 35th St (*)',
 'Western Ave & Congress Pkwy',
 'Oakley Ave & Irving Park Rd',
 'Humboldt Dr & Luis Munoz Marin Dr',
 'Cottage Grove Ave & 43rd St',
 'Halsted St & 21st St',
 'Kedzie Ave & 21st St',
 'Jeffery Blvd & 71st St',
 'Indiana Ave & 26th St',
 'Kosciuszko Park',
 'Damen Ave & Cullerton St',
 'Western Ave & 21st St',
 'Kedzie Ave & 24th St',
 'Ravenswood Ave & Balmoral Ave',
 'Seeley Ave & Roscoe St',
 'Cottage Grove Ave & 63rd St',
 'State St & 29th St',
 'Normal Ave & Archer Ave',
 'Wallace St & 35th St',
 'Halsted St & 47th Pl',
 'Western Ave & 24th St',
 'Shields Ave & 28th Pl',
 'Woodlawn Ave & Lake Park Ave',
 'Warren Park East',
 'Cottage Grove Ave & Oakwood Blvd',
 'California Ave & 21st St',
 'Clark St & Schiller St',
 'Prairie Ave & 43rd St',
 'Maplewood Ave & Peterson Ave',
 'Western Ave & Lunt Ave',
 'Princeton Ave & Garfield Blvd',
 'South Shore Dr & 74th St',
 'Stony Island Ave & 75th St',
 'MLK Jr Dr & 56th St (*)',
 'Wabash Ave & Cermak Rd',
 'Kedzie Ave & Harrison St',
 'Cottage Grove Ave & 51st St',
 'Spaulding Ave & Division St',
 'Central Park Ave & Ogden Ave',
 'Troy St & North Ave',
 'Clark St & Touhy Ave',
 'Budlong Woods Library',
 'Central Park Ave & North Ave',
 'Fairfield Ave & Roosevelt Rd',
 'Wood St & 35th St',
 'South Shore Dr & 67th St',
 'Shields Ave & 43rd St',
 'Warren Park West',
 'Pulaski Rd & Eddy St',
 'Western Ave & Granville Ave',
 'Indiana Ave & 40th St',
 'MLK Jr Dr & Oakwood Blvd',
 'Calumet Ave & 35th St',
 'Yates Blvd & 75th St',
 'Halsted St & 63rd St',
 'Princeton Ave & 47th St',
 'Ridge Blvd & Touhy Ave',
 'Cottage Grove Ave & 71st St',
 'Cottage Grove Ave & 67th St',
 'Albany Ave & 26th St',
 'Halsted St & 56th St',
 'Stony Island Ave & 71st St',
 'Sacramento Blvd & Franklin Blvd',
 'Wentworth Ave & 63rd St',
 'Saginaw Ave & Exchange Ave',
 'Perry Ave & 69th St',
 'Millard Ave & 26th St',
 'Union Ave & 42nd St',
 'Calumet Ave & 71st St',
 'Jeffery Blvd & 76th St',
 'Eberhart Ave & 61st St',
 'State St & Pershing Rd',
 'Lawndale Ave & 23rd St',
 'Halsted St & 69th St',
 'MLK Jr Dr & 63rd St',
 'Rainbow Beach',
 'Halsted St & Diversey Pkwy',
 'Halsted St & 51st St',
 'Benson Ave & Church St',
 'Central Ave & Madison St',
 'Oak Park Ave & Harrison St',
 'Chicago Ave & Washington St',
 'Laramie Ave & Kinzie St',
 'Damen Ave & Pershing Rd',
 'East Ave & Garfield St',
 'Ashland Ave & Pershing Rd',
 'Austin Blvd & Lake St',
 'Forest Ave & Lake St',
 'Cicero Ave & Lake St',
 'Oak Park Ave & South Blvd',
 'Kostner Ave & Adams St',
 'East Ave & Madison St',
 'Marion St & South Blvd',
 'Lombard Ave & Garfield St',
 'Pulaski Rd & Madison St',
 'Dodge Ave & Church St',
 'Ashland Ave & McDowell Ave',
 'Pulaski Rd & Congress Pkwy',
 'Cicero Ave & Flournoy St',
 'Kenton Ave & Madison St',
 'Kostner Ave & Lake St',
 'Conservatory Dr & Lake St',
 'Laramie Ave & Madison St',
 'Mason Ave & Madison St',
 'Ashland Ave & 63rd St',
 'Damen Ave & 61st St',
 'Western Blvd & 48th Pl',
 'Central St Metra',
 'Ashland Ave & 69th St',
 'Hoyne Ave & 47th St',
 'Ashland Ave & 66th St',
 'Wisconsin Ave & Madison St',
 'Ashland Ave & 50th St',
 'Lombard Ave & Madison St',
 'Central Park Blvd & 5th Ave',
 'Damen Ave & 51st St',
 'Morgan St & Pershing Rd',
 'Central Ave & Harrison St',
 'Cicero Ave & Quincy St',
 'Marshfield Ave & 59th St',
 'Pulaski Rd & Lake St',
 'Elizabeth St & 47th St',
 'Ashland Ave & Garfield Blvd',
 'Normal Ave & 72nd St',
 'Throop St & 52nd St',
 'State St & 76th St',
 'Laramie Ave & Gladys Ave',
 'Racine Ave & Garfield Blvd',
 'Albany (Kedzie) Ave & Montrose Ave',
 'Exchange Ave & 79th St',
 'Greenview Ave & Jarvis Ave',
 'Oakley Ave & Touhy Ave',
 'Central St & Girard Ave',
 'Clark St & Jarvis Ave',
 'Eastlake Ter & Rogers Ave',
 'Glenwood Ave & Touhy Ave',
 'Sheridan Rd & Noyes St (NU)',
 'University Library (NU)',
 'Paulina St & Howard St',
 'Knox Ave & Montrose Ave',
 'Ridgeland Ave & Lake St',
 'Field Museum',
 'Keystone Ave & Fullerton Ave',
 'Orleans St & Chestnut St (NEXT Apts)',
 'Wentworth Ave & 35th St',
 'Chicago Ave & Sheridan Rd',
 'Cuyler Ave & Augusta St',
 'Kilbourn Ave & Milwaukee Ave',
 'Milwaukee Ave & Cuyler Ave',
 'Stony Island Ave & 82nd St',
 'Kilbourn Ave & Irving Park Rd',
 'Forest Ave & Chicago Ave',
 'Bosworth Ave & Howard St',
 'Humphrey Ave & Ontario St',
 'Wolcott Ave & Fargo Ave',
 'Austin Blvd & Chicago Ave',
 'Avondale Ave & Irving Park Rd',
 'Ridge Blvd & Howard St',
 'Elmwood Ave & Austin St',
 'Cottage Grove Ave & 83rd St',
 'Chappel Ave & 79th St',
 'Central Ave & Chicago Ave',
 'State St & 79th St',
 'Western Ave & Howard St',
 'Ellis Ave & 83rd St',
 'Cottage Grove Ave & 78th St',
 'Valli Produce - Evanston Plaza',
 'Wabash Ave & 87th St',
 'Stony Island Ave & South Chicago Ave',
 'Vernon Ave & 75th St',
 'Commercial Ave & 83rd St',
 'Marshfield Ave & 44th St',
 'Wabash Ave & 83rd St',
 'Phillips Ave & 79th St',
 'South Chicago Ave & 83rd St',
 'May St & 69th St',
 'Vernon Ave & 79th St',
 'Evans Ave & 75th St',
 'Central Ave & Lake St',
 'Phillips Ave & 82nd St',
 'MLK Jr Dr & 83rd St',
 'Racine Ave & 65th St',
 'Greenwood Ave & 79th St',
 'Woodlawn Ave & 75th St',
 'Damen Ave & Garfield Blvd',
 'Halsted St & 59th St',
 'Racine Ave & 61st St',
 'Wood St & Hubbard St',
 'Sheffield Ave & Waveland Ave',
 'Clinton St & Jackson Blvd']

In [7]:
len(allSt16)

587

In [8]:
removed16 = list(set(allSt16) - set(allSt17))
len(removed16)

15

In [9]:
added17 = list(set(allSt17) - set(allSt16))
len(added17)

21

# Plotting all added stations of 2017 and removed stations of 2016

In [10]:
#reading the 'final_stations.csv' into a dataframe
#using all the stations co-ordinates and comparing with each of the year's stations added/ removed list and adding locations 
#     to each of the list to plot yearly data.

final = pd.read_csv('final_stations.csv')

In [11]:
final.head(1)

,Unnamed: 0,Station,Latitude,Longitude
0,0,Ogden Ave & Race Ave,41.891795,-87.658751


In [12]:
#Removing the unnecessary column in the dataframe
final = final.drop('Unnamed: 0', axis =1)

In [13]:
final.head(1)

,Station,Latitude,Longitude
0,Ogden Ave & Race Ave,41.891795,-87.658751


In [14]:
#Converting the dataframe into a dictionary
# key : Station name and value: latitude and longitude

locations = dict()

for station in final['Station']:
    locations[station] = {'lat': final[final['Station'] == station]['Latitude'].to_list()[0], 'long': final[final['Station'] == station]['Longitude'].to_list()[0]}

#printing out the first item in the dictionary to check the key-value pair
list(locations.items())[0]

('Ogden Ave & Race Ave', {'lat': 41.891795, 'long': -87.658751})

In [15]:
#Adding co-ordinates to all the stations that are added this year
n17 = {k: locations[k] for k in locations.keys() & set(added17)}

#printing out the first item in the dictionary to check the key-value pair
list(n17.items())[0]

('Eberhart (Vernon) Ave & 79th St',
 {'lat': 41.751345, 'long': -87.61256800000002})

In [16]:
#Converting the dictionary into dataframe
st17 = pd.DataFrame.from_dict(n17, orient='index')

In [17]:
#Resetting the index
st17.reset_index(inplace=True)

#Renaming the index column to Station name
st17 = st17.rename(columns = {'index':'Station Name'})

In [18]:
#Adding a column with values of color for every station
st17['color'] = st17.apply(lambda row: 'pink', axis=1)

#printing to check the structure of the dataframe
st17.head(1)

,Station Name,lat,long,color
0,Eberhart (Vernon) Ave & 79th St,41.751345,-87.612568,pink


In [19]:
removed16 = ['Wood St & Grand Ave',
 'Union Ave & Grand Ave',
 'Humboldt Dr & Luis Munoz Marin Dr',
 'Chappel Ave & 79th St',
 'Sheffield Ave & Addison St',
 'Damen Ave & Garfield Blvd',
 'Keeler Ave & Irving Park Rd',
 'Kedzie Ave & Montrose Ave',
 'Damen Ave & 61st St',
 'Union Ave & 42nd St',
 'Museum Campus',
 'Western Ave & Roscoe St',
 'Michigan Ave & Balbo Ave',
 'May St & Fulton St',
 'May St & Randolph St']

In [20]:
#adding co-ordinates for the removed stations in 2016
rm16 = {k: locations[k] for k in locations.keys() & set(removed16)}

#printing out the first item in the dictionary to check the key-value pair
list(rm16.items())[0]

('Kedzie Ave & Montrose Ave', {'lat': 41.96198976, 'long': -87.70816838})

In [21]:
#Converting the dictionary into dataframe
rm16 = pd.DataFrame.from_dict(rm16, orient='index')

In [22]:
#Resetting the index
rm16.reset_index(inplace=True)

#Renaming the index column to Station name
rm16 = rm16.rename(columns = {'index':'Station Name'})

In [23]:
#Adding a column with values of color for every station
rm16['color'] = rm16.apply(lambda row: 'red', axis=1)

#printing to check the structure of the dataframe
rm16.head(1)

,Station Name,lat,long,color
0,Kedzie Ave & Montrose Ave,41.96199,-87.708168,red


# Location Marker map for interactivity purpose (Station Name)

In [24]:
#merging both the added17 and removed16 dataframes
merged = pd.concat([rm16, st17], axis =0)

In [25]:
#setting marker as the color mentioned in the color column of the dataframe
marker = merged.loc[merged['color']!='']

map = folium.Map(location=[marker.lat.mean(), marker.long.mean()], zoom_start=12,
                     width=980,height=590, control_scale=True, tiles='OpenStreetMap')
marker.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]],
        popup=row['Station Name'],
          icon=folium.Icon(color=row['color'])
            ).add_to(map), axis=1)

map

# Circle marker map 2017
## Saving the map for converting into gif

In [38]:
#create a map
m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='cartodb positron')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2017')   
m.get_root().html.add_child(folium.Element(title_html))

#creates a CircleMarker and adds it to the map
def plotDot(point):
    folium.CircleMarker(location=[point.lat, point.long],             
                        color= "red",
                        radius=3,
                        weight=2
                       ).add_to(m)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
st17.apply(plotDot, axis = 1)

m

In [39]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [40]:
m.save('Gif_circlemarker/2017.html')

In [41]:
with fileinput.FileInput('Gif_circlemarker/2017.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [42]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_circlemarker')

fn='2017.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2017.png')
browser.quit()

In [43]:
image = Image.open('ss2017.png')
box = (0, 0, 1100, 700)
cropped_image = image.crop(box)
cropped_image.save('croppedss2017.png')

# Heatmap 2017
## Saving the map for converting into gif

In [32]:
numlist = st17.reset_index()[['lat', 'long']].values.tolist()


m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='OpenStreetMap')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2017')   
m.get_root().html.add_child(folium.Element(title_html))


HeatMap(numlist).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m

In [33]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [34]:
m.save('Gif_heatmap/2017.html')

In [35]:
with fileinput.FileInput('Gif_heatmap/2017.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [36]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_heatmap')

fn='2017.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2017.png')
browser.quit()

In [37]:
image = Image.open('ss2017.png')
box = (0, 0, 1250, 780)
cropped_image = image.crop(box)
cropped_image.save('croppedss2017.png')